You'll be using the Dark Sky Forecast API from Forecast.io, available at https://developer.forecast.io. It's a pretty simple API, but be sure to read the documentation!

A forecast request returns the current forecast (for the next week): 
https://api.forecast.io/forecast/APIKEY/LATITUDE,LONGITUDE

A time-machine request returns the observed weather at a given time (for many places, up to 60 years in the past): https://api.forecast.io/forecast/APIKEY/LATITUDE,LONGITUDE,TIME


1) Make a request from the Forecast.io API for where you were born (or lived, or want to visit!).

In [40]:
import requests

In [41]:
response = requests.get("https://api.forecast.io/forecast/e554f37a8164ce189acd210d00a452e0/47.4079,9.4647")
weather_data = response.json()
weather_data.keys()

dict_keys(['longitude', 'timezone', 'daily', 'currently', 'latitude', 'flags', 'hourly', 'offset'])

In [43]:
print(weather_data['timezone'])

Europe/Zurich


The time zone is correct!

In [44]:
print("Longitude:", weather_data['longitude'], "Latitude", weather_data['latitude'])

Longitude: 9.4647 Latitude 47.4079


The longitude is mentioned first, and then the latitude. Usually, it is the other way round.

2) What's the current wind speed? How much warmer does it feel than it actually is?

In [45]:
type(weather_data['currently'])

dict

In [46]:
weather_data['currently'].keys()

dict_keys(['icon', 'precipIntensity', 'ozone', 'precipType', 'cloudCover', 'apparentTemperature', 'visibility', 'temperature', 'precipProbability', 'humidity', 'pressure', 'windSpeed', 'windBearing', 'dewPoint', 'time', 'summary'])

In [47]:
print("The wind in Trogen, Switzerland, is currently blowing at", weather_data['currently']['windSpeed'], "mph.")

The wind in Trogen, Switzerland, is currently blowing at 2.93 mph.


In [58]:
weather = weather_data['currently']
Temperature = int(weather['apparentTemperature']) - int(weather['temperature'])
Celsius = round((int(weather['temperature']) - 32) * 5 / 9)
if Temperature == 0:
    print("The temperature feels exactly as you would expect it to feel, namely", weather['temperature'], "degrees Fahrenheit, or", Celsius, "degrees Celsius.")
elif Temperature > 0:
    print("It feels", Temperature, "degrees Fahrenheit warmer than the actual temperature, which is", weather['temperature'], "degrees Fahrenheit, or", Celsius, "degrees Celsius.")
else:
    print("It feels", Temperature, "degrees Fahrenheit colder than the actual temperature, which is", weather['temperature'], "degrees Fahrenheit, or", Celsius, "degrees Celsius.")

The temperature feels exactly as you would expect it to feel, namely 52.7 degrees Fahrenheit, or 11 degrees Celsius.


3) The first daily forecast is the forecast for today. For the place you decided on up above, how much of the moon is currently visible?

In [77]:
type(weather_data['daily'])

dict

In [78]:
weather_data['daily'].keys()

dict_keys(['data', 'icon', 'summary'])

In [79]:
type(weather_data['daily']['data'])

list

In [90]:
for phase in weather_data['daily']['data']:
    moon_forecast_tomorrow = phase['moonPhase']
    break
#print(phase['moonPhase'])
if moon_forecast_tomorrow == 0:
    print("Tomorrow is New Moon.")
elif moon_forecast_tomorrow > .75:
    print("Tomorrow is the Moon is in a Waning Crescent phase.")
elif moon_forecast_tomorrow == .75:
    print("Tomorrow is last quarter Moon.")
elif moon_forecast_tomorrow > .5:
    print("Tommorrow is the Moon is in a Waning Gibbous phase.")
elif moon_forecast_tomorrow == .5:
    print("Tommorrow is Full Moon.")
elif moon_forecast_tomorrow > .25:
    print("Tommorrow is the Moon is a Waxing Gibbous phase.")
elif moon_forecast_tomorrow == .25:
    print("Tommorrow is first Quarter Moon.")
elif moon_forecast_tomorrow > 0:
    print("Tommorrow is the Moon is in a Waxing Crescent phase. This is the first phase after New Moon.")


Tommorrow is the Moon is in a Waxing Crescent phase. This is the first phase after New Moon.


4) What's the difference between the high and low temperatures for today?

In [101]:
print(weather_data['currently'])

{'icon': 'rain', 'precipIntensity': 0.0225, 'ozone': 347.15, 'precipType': 'rain', 'cloudCover': 0.79, 'apparentTemperature': 52.7, 'visibility': 6.21, 'temperature': 52.7, 'precipProbability': 0.56, 'humidity': 0.91, 'pressure': 1017.36, 'windSpeed': 2.93, 'windBearing': 283, 'dewPoint': 50.08, 'time': 1465429364, 'summary': 'Light Rain'}


In [102]:
response = requests.get("https://api.forecast.io/forecast/e554f37a8164ce189acd210d00a452e0/47.4079,9.4647,1465420000")
hist_weather_data = response.json()

hist_weather_data.keys()




dict_keys(['longitude', 'timezone', 'daily', 'currently', 'latitude', 'flags', 'hourly', 'offset'])

In [103]:
hist_weather_data['daily'].keys()

dict_keys(['data'])

In [104]:
print(hist_weather_data['daily']['data'])

[{'precipIntensityMax': 0.0488, 'temperatureMax': 65.49, 'sunriseTime': 1465356438, 'temperatureMin': 53.83, 'precipIntensityMaxTime': 1465408800, 'precipType': 'rain', 'apparentTemperatureMin': 53.83, 'visibility': 6.14, 'moonPhase': 0.12, 'precipIntensity': 0.0157, 'humidity': 0.77, 'pressure': 1019.56, 'windSpeed': 2.35, 'windBearing': 254, 'apparentTemperatureMax': 65.49, 'temperatureMinTime': 1465351200, 'sunsetTime': 1465413482, 'ozone': 343.66, 'temperatureMaxTime': 1465390800, 'apparentTemperatureMinTime': 1465351200, 'time': 1465336800, 'apparentTemperatureMaxTime': 1465390800, 'icon': 'rain', 'cloudCover': 0.42, 'precipProbability': 0.65, 'summary': 'Light rain throughout the day.', 'dewPoint': 52.11}]


In [112]:
for n in hist_weather_data['daily']['data']:
    Min = n['temperatureMin']
    Max = n['temperatureMax']

In [114]:
Min_Max = Max - Min
Min_Max_Celsius = ((Max - 32) * 5 / 9) - ((Min - 32) * 5 / 9)
print("The diffrence between the high and low temperatures on Wendesday, June 8, in Trogen, Switzerland, was", round(Min_Max), "Fahrenheit", "or", round(Min_Max_Celsius), "Celsius.")

The diffrence between the high and low temperatures on Wendesday, June 8, was 12 Fahrenheit or 6 Celsius.


5) Loop through the daily forecast, printing out the next week's worth of predictions. I'd like to know the high temperature for each day, and whether it's hot, warm, or cold, based on what temperatures you think are hot, warm or cold.

In [158]:
response = requests.get("https://api.forecast.io/forecast/e554f37a8164ce189acd210d00a452e0/40.7128,-74.0059")
weather_data = response.json()
weather_data.keys()

dict_keys(['longitude', 'timezone', 'minutely', 'daily', 'currently', 'latitude', 'flags', 'hourly', 'offset'])

In [159]:
weather_data['daily'].keys()

dict_keys(['data', 'icon', 'summary'])

In [160]:
print(weather_data['daily']['data'][0])

{'precipIntensityMax': 0.0974, 'temperatureMax': 66.27, 'sunriseTime': 1465377971, 'temperatureMin': 56.91, 'precipIntensityMaxTime': 1465408800, 'precipType': 'rain', 'apparentTemperatureMin': 56.91, 'visibility': 9.29, 'moonPhase': 0.13, 'precipIntensity': 0.0102, 'humidity': 0.63, 'pressure': 1004.47, 'windSpeed': 8.84, 'windBearing': 285, 'apparentTemperatureMax': 66.27, 'temperatureMinTime': 1465408800, 'sunsetTime': 1465432021, 'ozone': 369.45, 'temperatureMaxTime': 1465398000, 'apparentTemperatureMinTime': 1465408800, 'time': 1465358400, 'apparentTemperatureMaxTime': 1465398000, 'icon': 'rain', 'cloudCover': 0.22, 'precipProbability': 0.72, 'summary': 'Rain in the afternoon.', 'dewPoint': 48.53}


In [161]:
Forecast = weather_data['daily']['data']
for max_temp in Forecast:
    Convert_Celsius = (max_temp['temperatureMax'] - 32) * 5 / 9
    if Convert_Celsius > 30:
        Temperature = "hot"
    elif Convert_Celsius > 20:
        Temperature = "warm"
    else:
        Temperature = "cold"
    print("The maximum temperature for next week are", max_temp['temperatureMax'], "Fahrenheit or", round(Convert_Celsius), "Celsius. That is", Temperature)

The maximum temperature for next week are 66.27 Fahrenheit or 19 Celsius. That is cold
The maximum temperature for next week are 74.06 Fahrenheit or 23 Celsius. That is warm
The maximum temperature for next week are 76.17 Fahrenheit or 25 Celsius. That is warm
The maximum temperature for next week are 75.09 Fahrenheit or 24 Celsius. That is warm
The maximum temperature for next week are 79.38 Fahrenheit or 26 Celsius. That is warm
The maximum temperature for next week are 68.88 Fahrenheit or 20 Celsius. That is warm
The maximum temperature for next week are 75.06 Fahrenheit or 24 Celsius. That is warm
The maximum temperature for next week are 76.72 Fahrenheit or 25 Celsius. That is warm


In [162]:
 import datetime
print(datetime.datetime.fromtimestamp(int("1284101485")).strftime('%Y-%m-%d'))

2010-09-10


In [163]:
import time
import datetime
Forecast = weather_data['daily']['data']
for max_temp in Forecast:
    Convert_Celsius = (max_temp['temperatureMax'] - 32) * 5 / 9
    time = max_temp['time']
    if Convert_Celsius > 30:
        Temperature = "hot"
    elif Convert_Celsius > 20:
        Temperature = "warm"
    else:
        Temperature = "cold"
    print("On", datetime.datetime.fromtimestamp(int(time)).strftime('%Y-%m-%d'), "the maximum temperature will be", max_temp['temperatureMax'], "Fahrenheit or", round(Convert_Celsius), "Celsius. That is", Temperature)

On 2016-06-08 the maximum temperature will be 66.27 Fahrenheit or 19 Celsius. That is cold
On 2016-06-09 the maximum temperature will be 74.06 Fahrenheit or 23 Celsius. That is warm
On 2016-06-10 the maximum temperature will be 76.17 Fahrenheit or 25 Celsius. That is warm
On 2016-06-11 the maximum temperature will be 75.09 Fahrenheit or 24 Celsius. That is warm
On 2016-06-12 the maximum temperature will be 79.38 Fahrenheit or 26 Celsius. That is warm
On 2016-06-13 the maximum temperature will be 68.88 Fahrenheit or 20 Celsius. That is warm
On 2016-06-14 the maximum temperature will be 75.06 Fahrenheit or 24 Celsius. That is warm
On 2016-06-15 the maximum temperature will be 76.72 Fahrenheit or 25 Celsius. That is warm


In [ ]:
import datetime
print(datetime.datetime.fromtimestamp(int(time)).strftime('%Y-%m-%d'))

6) What's the weather looking like for the rest of today in Miami, Florida? I'd like to know the temperature for every hour, and if it's going to have cloud cover of more than 0.5 say "{temperature} and cloudy" instead of just the temperature.

In [185]:
response = requests.get("https://api.forecast.io/forecast/e554f37a8164ce189acd210d00a452e0/25.7617,-80.1918")
Florida = response.json()
Florida.keys()

dict_keys(['longitude', 'timezone', 'minutely', 'daily', 'currently', 'latitude', 'flags', 'hourly', 'offset'])

In [186]:
Florida['hourly'].keys()

dict_keys(['data', 'icon', 'summary'])

In [203]:
#print(Florida['hourly']['data'])

In [221]:
import time
import datetime
Hourly = Florida['hourly']['data']
Hourly = Hourly
for weather in Hourly:
    time = weather['time']
    if weather['cloudCover'] > 0.5:
        cloudy = "and cloudy"
    else:
        cloudy = "not cloudy"
    print(datetime.datetime.fromtimestamp(int(time)).strftime('%H:%M'), "{", weather['temperature'], "°F}",cloudy)

22:00 { 76.8 °F} and cloudy
23:00 { 76.56 °F} and cloudy
00:00 { 76.65 °F} and cloudy
01:00 { 76.47 °F} and cloudy
02:00 { 76.78 °F} and cloudy
03:00 { 77.11 °F} and cloudy
04:00 { 77.83 °F} and cloudy
05:00 { 78.16 °F} and cloudy
06:00 { 77.88 °F} and cloudy
07:00 { 78 °F} and cloudy
08:00 { 78.86 °F} and cloudy
09:00 { 80.1 °F} and cloudy
10:00 { 81.81 °F} and cloudy
11:00 { 83.69 °F} and cloudy
12:00 { 84.97 °F} and cloudy
13:00 { 85.9 °F} and cloudy
14:00 { 86.58 °F} and cloudy
15:00 { 86.04 °F} and cloudy
16:00 { 85.77 °F} and cloudy
17:00 { 85.25 °F} and cloudy
18:00 { 84.46 °F} and cloudy
19:00 { 83.37 °F} and cloudy
20:00 { 82.84 °F} and cloudy
21:00 { 82.04 °F} and cloudy
22:00 { 81.33 °F} and cloudy
23:00 { 81.13 °F} and cloudy
00:00 { 80.76 °F} and cloudy
01:00 { 80.21 °F} and cloudy
02:00 { 79.54 °F} and cloudy
03:00 { 79 °F} and cloudy
04:00 { 78.41 °F} and cloudy
05:00 { 77.88 °F} and cloudy
06:00 { 77.54 °F} and cloudy
07:00 { 78.38 °F} and cloudy
08:00 { 79.75 °F} and c

7) What was the temperature in Central Park on Christmas Day, 1980? How about 1990? 2000?

In [183]:
response = requests.get("https://api.forecast.io/forecast/e554f37a8164ce189acd210d00a452e0/40.781750,-73.966641,346593600")
weather_data = response.json()
weather_data.keys()

dict_keys(['longitude', 'timezone', 'daily', 'currently', 'latitude', 'flags', 'hourly', 'offset'])

In [170]:
#print(weather_data['daily']['data'][0])

In [169]:
for Christmas in weather_data['daily']['data']:
    Convert_Celsius = (Christmas['temperatureMax'] - 32) * 5 / 9
    print("The maximum temperature on Christmas Day 1980 in Centralpark was", 
          Christmas['temperatureMax'], "Fahrenheit, or", round(Convert_Celsius), "degrees Celsius.")

The maximum temperature on Christmas Day 1980 in Centralpark was 29.53 Fahrenheit, or -1 degrees.


In [172]:
response = requests.get("https://api.forecast.io/forecast/e554f37a8164ce189acd210d00a452e0/40.781750,-73.966641,662126400")
weather_data_1990 = response.json()
weather_data_1990.keys()

response = requests.get("https://api.forecast.io/forecast/e554f37a8164ce189acd210d00a452e0/40.781750,-73.966641,977745600")
weather_data_2000 = response.json()
weather_data_2000.keys()

dict_keys(['longitude', 'timezone', 'daily', 'currently', 'latitude', 'flags', 'hourly', 'offset'])

In [173]:
for Christmas in weather_data_1990['daily']['data']:
    Convert_Celsius = (Christmas['temperatureMax'] - 32) * 5 / 9
    print("The maximum temperature on Christmas Day 1980 in Centralpark was", 
          Christmas['temperatureMax'], "Fahrenheit, or", round(Convert_Celsius), "degrees Celsius.")

The maximum temperature on Christmas Day 1980 in Centralpark was 32.02 Fahrenheit, or 0 degrees Celsius.


In [174]:
for Christmas in weather_data_2000['daily']['data']:
    Convert_Celsius = (Christmas['temperatureMax'] - 32) * 5 / 9
    print("The maximum temperature on Christmas Day 1980 in Centralpark was", 
          Christmas['temperatureMax'], "Fahrenheit, or", round(Convert_Celsius), "degrees Celsius.")

The maximum temperature on Christmas Day 1980 in Centralpark was 27.02 Fahrenheit, or -3 degrees Celsius.
